<a href="https://colab.research.google.com/github/tmu-nlp/100knock2021/blob/main/wei/chapter08/knock72.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task Description
72. 損失と勾配の計算

学習データの事例$x_1$と事例集合$x_1$,$x_2$,$x_3$,$x_4$に対して，CrossEntropyLossと，行列$W$に対する勾配を計算せよ．なお，ある事例$x_i$に対して損失は次式で計算される．
$$l_i=−log[事例x_iがy_iに分類される確率]$$


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読込
df = pd.read_csv('drive/MyDrive/ColabNotebooks/NLPknock100/newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# データの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

# データの分割
train, valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=valid_test['CATEGORY'])

In [2]:
from gensim.models import KeyedVectors
import string
import torch

# 学習済み単語ベクトルを読み込む
model = KeyedVectors.load_word2vec_format('drive/MyDrive/ColabNotebooks/NLPknock100/GoogleNews-vectors-negative300.bin.gz', binary=True)

def transform_w2v(text):
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  words = text.translate(table).split()  # 記号をスペースに置換後、スペースで分割してリスト化
  vec = [model[word] for word in words if word in model]  # 1語ずつベクトル化

  return torch.tensor(sum(vec) / len(vec))  # 平均ベクトルをTensor型に変換して出力

X_train = torch.stack([transform_w2v(text) for text in train['TITLE']])
X_valid = torch.stack([transform_w2v(text) for text in valid['TITLE']])
X_test = torch.stack([transform_w2v(text) for text in test['TITLE']])

In [3]:
# ラベルベクトルの作成
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
y_train = torch.tensor(train['CATEGORY'].map(lambda x: category_dict[x]).values)
y_valid = torch.tensor(valid['CATEGORY'].map(lambda x: category_dict[x]).values)
y_test = torch.tensor(test['CATEGORY'].map(lambda x: category_dict[x]).values)

In [4]:
# SGLNetという単層ニューラルネットワークを定義
from torch import nn

class SGLNet(nn.Module):
  #　ネットのlayerを定義
  def __init__(self, input_size, output_size):
    super().__init__()
    self.fc = nn.Linear(input_size, output_size, bias=False)
    nn.init.normal_(self.fc.weight, 0.0, 1.0)  # 正規乱数で重みを初期化
  #　forwardで入力データが順伝播時に通るレイヤーを順に配置しておく
  def forward(self, x):
    x = self.fc(x)
    return x

In [5]:
# 単層ニューラルネットワークの初期化
SigelNNmodel = SGLNet(300, 4) 

In [6]:
# 学習用のTensor型の平均化ベクトルとラベルベクトルを入力することで、集合にある各事例の平均損失を計算
# 入力ベクトルはsoftmax前の値
criterion = nn.CrossEntropyLoss()
l_1 = criterion(SigelNNmodel(X_train[:1]), y_train[:1])  
SigelNNmodel.zero_grad()  # 勾配をゼロで初期化
l_1.backward()  # 勾配を計算
print(f'損失: {l_1:.4f}')
print(f'勾配:\n{SigelNNmodel.fc.weight.grad}')

損失: 2.0449
勾配:
tensor([[-0.0728, -0.0049, -0.0059,  ..., -0.0654, -0.0377,  0.0756],
        [ 0.0307,  0.0021,  0.0025,  ...,  0.0276,  0.0159, -0.0318],
        [ 0.0016,  0.0001,  0.0001,  ...,  0.0014,  0.0008, -0.0017],
        [ 0.0405,  0.0027,  0.0033,  ...,  0.0364,  0.0210, -0.0421]])


In [7]:
#@title
l = criterion(SigelNNmodel(X_train[:4]), y_train[:4])
SigelNNmodel.zero_grad()
l.backward()
print(f'損失: {l:.4f}')
print(f'勾配:\n{SigelNNmodel.fc.weight.grad}')

損失: 1.3052
勾配:
tensor([[-0.0123,  0.0027, -0.0166,  ..., -0.0330, -0.0005,  0.0256],
        [ 0.0021, -0.0044,  0.0196,  ...,  0.0274,  0.0155, -0.0055],
        [ 0.0094, -0.0062, -0.0111,  ..., -0.0005, -0.0001,  0.0053],
        [ 0.0008,  0.0079,  0.0081,  ...,  0.0061, -0.0149, -0.0254]])
